# Arrow/Parquet Challenge

In [ ]:
pip install awswrangler

In [6]:
import awswrangler as wr

path = f"s3://unravel-saas-demo/test/sales.parquet"
data = wr.s3.read_parquet(path)

print(data)

   account    product        date  quantity
0        1      apple  1631138217         9
1        2     banana  1631208217         9
2        3  pineapple  1631308217        11
3        4       plum  1631378217         8
4        5     orange  1631478217        11
5        6      guava  1631648217         3
6        7       kiwi  1631548217        14


In [58]:
def analyze(df):
    df['date'] = pd.to_datetime(df['date'], unit='s')
    print(df)
    print("------------\n")
    s = df['date']
    print(s)
    print("------------\n")
    #s1 = pd.date_range(s[0], s[4], freq='D').to_series()
    print(df['date'].dt.dayofweek)
    print("------------\n")
    print(df['date'].dt.day_name())

    print("------------\n")
    sum_of_quantity = df['quantity'].sum()
    print(f"sum of quantity: {sum_of_quantity}")

    print("------------\n")
    var_of_quantity = df.loc[:, "quantity"].var()
    print(f"variance of quantity: {var_of_quantity}")

    accounts = df['account'].unique()
    unique_accounts = pd.Series(accounts)
    print("------------\n")
    print(type(unique_accounts))
    #print(f"unique accounts: {unique_accounts}")
    for account, value in unique_accounts.items():
        print(f"account #: {account}, Value: {value}")

    print("------------\n")
    products = df['product'].unique()
    unique_products = pd.Series(products)
    #print(f"unique products: {unique_products}")
    for product, value in unique_products.items():
        print(f"product #: {product}, Value: {value}")

    print("------------\n")
    row_count = len(df.index)
    print(f"Number of rows: {row_count}")


## Compute

In [59]:
import pandas as pd

analyze(data)

   account    product                date  quantity
0        1      apple 2021-09-08 21:56:57         9
1        2     banana 2021-09-09 17:23:37         9
2        3  pineapple 2021-09-10 21:10:17        11
3        4       plum 2021-09-11 16:36:57         8
4        5     orange 2021-09-12 20:23:37        11
5        6      guava 2021-09-14 19:36:57         3
6        7       kiwi 2021-09-13 15:50:17        14
------------

0   2021-09-08 21:56:57
1   2021-09-09 17:23:37
2   2021-09-10 21:10:17
3   2021-09-11 16:36:57
4   2021-09-12 20:23:37
5   2021-09-14 19:36:57
6   2021-09-13 15:50:17
Name: date, dtype: datetime64[ns]
------------

0    2
1    3
2    4
3    5
4    6
5    1
6    0
Name: date, dtype: int64
------------

0    Wednesday
1     Thursday
2       Friday
3     Saturday
4       Sunday
5      Tuesday
6       Monday
Name: date, dtype: object
------------

sum of quantity: 65
------------

variance of quantity: 11.571428571428571
------------

<class 'pandas.core.series.Serie

In [14]:
import pyarrow as pa
import pyarrow.parquet as pq

from pyarrow import fs

In [23]:
record_batch = pa.RecordBatch.from_pandas(data)
print(record_batch)


pyarrow.RecordBatch
account: int64
product: string
date: timestamp[ns]
quantity: int64


In [65]:
def get_parquet_metadata():
    table = pq.read_table(path)
    return table
    
'''
    pq_file = pq.ParquetFile(path)
    print(pq_file.metadata)
    print(pq_file.schema)
    print(pq_file.metadata.row_group(0))
    print(pq_file.metadata.row_group(0).column(0))
'''    


'\n    pq_file = pq.ParquetFile(path)\n    print(pq_file.metadata)\n    print(pq_file.schema)\n    print(pq_file.metadata.row_group(0))\n    print(pq_file.metadata.row_group(0).column(0))\n'

In [66]:
table = get_parquet_metadata()

print(table)

pyarrow.Table
account: int64
product: string
date: int64
quantity: int64


In [38]:
batch = pa.Table.to_batches(table)
print(batch)

[pyarrow.RecordBatch
account: int64
product: string
date: int64
quantity: int64]


In [46]:
mean = pa.compute.mean(table[3])
print(f"mean of quantity: {mean}")

mean of quantity: 9.285714285714286


## Update column data

In [47]:
print(table[3])

[
  [
    9,
    9,
    11,
    8,
    11,
    3,
    14
  ]
]


In [52]:
col_quantity = pa.array([10, 9, 8, 7, 6, 5, 4])
t1 = pa.Table.set_column(table, 3, 'quantity', col_quantity)

print(t1)

pyarrow.Table
account: int64
product: string
date: int64
quantity: int64


In [53]:
print(t1[3])

[
  [
    10,
    9,
    8,
    7,
    6,
    5,
    4
  ]
]


In [70]:
#t2 = pa.Table.from_batches(table)

#t_all = pa.concat_tables(table, )

for i in table:
    print(i)

[
  [
    1,
    2,
    3,
    4,
    5,
    6,
    7
  ]
]
[
  [
    "apple",
    "banana",
    "pineapple",
    "plum",
    "orange",
    "guava",
    "kiwi"
  ]
]
[
  [
    1631138217,
    1631208217,
    1631308217,
    1631378217,
    1631478217,
    1631648217,
    1631548217
  ]
]
[
  [
    9,
    9,
    11,
    8,
    11,
    3,
    14
  ]
]


In [98]:
print(len(table.columns))

l = []

for x in table:
    l.append(pa.ChunkedArray.iterchunks(x))

for x in t1:
    l.append(pa.ChunkedArray.iterchunks(x))
    
print(len(l))

4
8


In [100]:
path = f"s3://unravel-saas-demo/test/arrow_test.parquet"

In [101]:
def read_from_s3():
    return wr.s3.read_parquet([path])

In [102]:
s3_data_df = read_from_s3()
print(s3_data_df)

   account     product        date  quantity
0        1       apple  1631734436        10
1        2      banana  1631664436         4
2        3       grape  1631534436        14
3        4   pineapple  1631434436        10
4        5       guava  1631334436        10
5        6  watermelon  1631234436         4
6        7        plum  1631134436        16


## Computations

In [110]:
# Get field value count
count = pa.compute.count(table[0])
print(count)

7


In [112]:
# Get mean of array data
mean = pa.compute.mean(table[0])
print(mean)

4.0


In [113]:
# Get sum of 'Quantity' field
sum = pa.compute.sum(table[3])
print(sum)

65


In [114]:
# Get variance of quantity
var = pa.compute.variance(table[3])
print(var)

9.918367346938776


In [118]:
# Get number of unique account
uniq_account = pa.compute.unique(table[0])
print(uniq_account)

[
  1,
  2,
  3,
  4,
  5,
  6,
  7
]


In [117]:
# Get number of unique product
uniq_product = pa.compute.unique(table[1])
print(uniq_product)

[
  "apple",
  "banana",
  "pineapple",
  "plum",
  "orange",
  "guava",
  "kiwi"
]
